<a href="https://colab.research.google.com/github/Hongzf/OLID/blob/main/OLID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install emoji
!pip install wordsegment
!pip install transformers
!pip install trainer
!pip install attention

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython.display import HTML
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import r2_score

from sklearn.metrics import confusion_matrix

import os
import emoji
from wordsegment import load, segment
import pickle
import torch
from torch.utils.data import Dataset
import random

from transformers import BertTokenizer, RobertaTokenizer, get_cosine_schedule_with_warmup
from trainer import Trainer 
from torch import nn
from transformers import BertModel, BertForSequenceClassification, RobertaForSequenceClassification, RobertaModel
from attention import Attention
load()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
#optional Google drive integration
from google.colab import drive
drive.mount('/content/drive')
#path1 = "./drive/MyDrive/OLID/testset-levela.tsv"
#path2 = "./drive/MyDrive/OLID/labels-levelb.csv"
#path3 = "./drive/MyDrive/OLID/labels-levelc.csv"

Mounted at /content/drive


In [ ]:

#df = pd.read_csv(path1, sep='\t', keep_default_na=False)
#df.info()

In [ ]:
def save(toBeSaved, filename, mode='wb'):
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    file = open(filename, mode)
    pickle.dump(toBeSaved, file)
    file.close()

def load(filename, mode='rb'):
    file = open(filename, mode)
    loaded = pickle.load(file)
    file.close()
    return loaded

def pad_sents(sents, pad_token):
    sents_padded = []
    lens = get_lens(sents)
    max_len = max(lens)
    sents_padded = [sents[i] + [pad_token] * (max_len - l) for i, l in enumerate(lens)]
    return sents_padded

def sort_sents(sents, reverse=True):
    sents.sort(key=(lambda s: len(s)), reverse=reverse)
    return sents

def get_mask(sents, unmask_idx=1, mask_idx=0):
    lens = get_lens(sents)
    max_len = max(lens)
    mask = [([unmask_idx] * l + [mask_idx] * (max_len - l)) for l in lens]
    return mask

def get_lens(sents):
    return [len(sent) for sent in sents]

def get_max_len(sents):
    max_len = max([len(sent) for sent in sents])
    return max_len

def truncate_sents(sents, length):
    sents = [sent[:length] for sent in sents]
    return sents

def get_loss_weight(labels, label_order):
    nums = [np.sum(labels == lo) for lo in label_order]
    loss_weight = torch.tensor([n / len(labels) for n in nums])
    return loss_weight

In [ ]:
""" Define the Attention Layer of the model.
"""

from __future__ import print_function, division

import torch

from torch.autograd import Variable
from torch.nn import Module
from torch.nn.parameter import Parameter

class Attention(Module):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, attention_size, return_attention=False):
        """ Initialize the attention layer
        # Arguments:
            attention_size: Size of the attention vector.
            return_attention: If true, output will include the weight for each input token
                              used for the prediction
        """
        super(Attention, self).__init__()
        self.return_attention = return_attention
        self.attention_size = attention_size
        self.attention_vector = Parameter(torch.FloatTensor(attention_size))
        self.attention_vector.data.normal_(std=0.05) # Initialize attention vector

        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    def __repr__(self):
        s = '{name}({attention_size}, return attention={return_attention})'
        return s.format(name=self.__class__.__name__, **self.__dict__)

    def forward(self, inputs, input_lengths):
        """ Forward pass.
        # Arguments:
            inputs (Torch.Variable): Tensor of input sequences
            input_lengths (torch.LongTensor): Lengths of the sequences
        # Return:
            Tuple with (representations and attentions if self.return_attention else None).
        """
        logits = inputs.matmul(self.attention_vector)
        unnorm_ai = (logits - logits.max()).exp()

        # Compute a mask for the attention on the padded sequences
        # See e.g. https://discuss.pytorch.org/t/self-attention-on-words-and-masking/5671/5
        max_len = unnorm_ai.size(1)
        idxes = torch.arange(0, max_len, out=torch.LongTensor(max_len)).unsqueeze(0).to(device=self.device)
        mask = Variable((idxes < input_lengths.unsqueeze(1)).float())

        # apply mask and renormalize attention scores (weights)
        masked_weights = unnorm_ai * mask
        att_sums = masked_weights.sum(dim=1, keepdim=True)  # sums per sequence
        attentions = masked_weights.div(att_sums)

        # apply attention weights
        weighted = torch.mul(inputs, attentions.unsqueeze(-1).expand_as(inputs))

        # get the final fixed vector representations of the sentences
        representations = weighted.sum(dim=1)

        return (representations, attentions if self.return_attention else None)

In [ ]:
OLID_PATH = './drive/MyDrive/OLID'
SAVE_PATH = '.drive/MyDrive/OLID/save'
LABEL_DICT = {
    'a': {'OFF': 0, 'NOT': 1},
    'b': {'TIN': 0, 'UNT': 1, 'NULL': 2},
    'c': {'IND': 0, 'GRP': 1, 'OTH': 2, 'NULL': 3}
}

TRAIN_PATH = os.path.join(OLID_PATH, 'olid-training-v1.0.tsv')

Load Data

In [ ]:
def read_file(filepath: str):
    df = pd.read_csv(filepath, sep='\t', keep_default_na=False)
    # df= df.sample(frac = 0.9)
    ids = np.array(df['id'].values)
    tweets = np.array(df['tweet'].values)
 
    # Process tweets
    tweets = process_tweets(tweets)

    label_a = np.array(df['subtask_a'].values)
    label_b = df['subtask_b'].values
    label_c = np.array(df['subtask_c'].values)
    nums = len(df)

    return nums, ids, tweets, label_a, label_b, label_c

def read_test_file(task, tokenizer, truncate=512):
    df1 = pd.read_csv(os.path.join(OLID_PATH, 'testset-level' + task + '.tsv'), sep='\t')
    df2 = pd.read_csv(os.path.join(OLID_PATH, 'labels-level' + task + '.csv'), sep=',')
    # df1 = df1.sample(frac = 0.9)
    # df2 = df2.sample(frac = 0.9)
    ids = np.array(df1['id'].values)
    tweets = np.array(df1['tweet'].values)
    labels = np.array(df2['label'].values)
    nums = len(df1)

    # Process tweets
    tweets = process_tweets(tweets)

    # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    token_ids = [tokenizer.encode(text=tweets[i], add_special_tokens=True, max_length=truncate) for i in range(nums)]
    mask = np.array(get_mask(token_ids))
    lens = get_lens(token_ids)
    token_ids = np.array(pad_sents(token_ids, tokenizer.pad_token_id))

    return ids, token_ids, lens, mask, labels

def read_test_file_all(tokenizer, truncate=512):
    df = pd.read_csv(os.path.join(OLID_PATH, 'testset-levela.tsv'), sep='\t')
    df_a = pd.read_csv(os.path.join(OLID_PATH, 'labels-levela.csv'), sep=',')
    ids = np.array(df['id'].values)
    tweets = np.array(df['tweet'].values)
    label_a = np.array(df_a['label'].values)
    nums = len(df)

    # Process tweets
    tweets = process_tweets(tweets)

    df_b = pd.read_csv(os.path.join(OLID_PATH, 'labels-levelb.csv'), sep=',')
    df_c = pd.read_csv(os.path.join(OLID_PATH, 'labels-levelc.csv'), sep=',')
    label_data_b = dict(zip(df_b['id'].values, df_b['label'].values))
    label_data_c = dict(zip(df_c['id'].values, df_c['label'].values))
    label_b = [label_data_b[id] if id in label_data_b.keys() else 'NULL' for id in ids]
    label_c = [label_data_c[id] if id in label_data_c.keys() else 'NULL' for id in ids]

    token_ids = [tokenizer.encode(text=tweets[i], add_special_tokens=True, max_length=truncate) for i in range(nums)]
    mask = np.array(get_mask(token_ids))
    lens = get_lens(token_ids)
    token_ids = np.array(pad_sents(token_ids, tokenizer.pad_token_id))

    return ids, token_ids, lens, mask, label_a, label_b, label_c

In [ ]:
def task_a(filepath: str, tokenizer, truncate=512):
    nums, ids, tweets, label_a, _, _ = read_file(filepath)
    # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    token_ids = [tokenizer.encode(text=tweets[i], add_special_tokens=True, max_length=truncate) for i in range(nums)]
    mask = np.array(get_mask(token_ids))
    lens = get_lens(token_ids)
    token_ids = np.array(pad_sents(token_ids, tokenizer.pad_token_id))

    return ids, token_ids, lens, mask, label_a

def task_b(filepath: str, tokenizer, truncate=512):
    nums, ids, tweets, _, label_b, _ = read_file(filepath)
    # Only part of the tweets are useful for task b

    useful = label_b != 'NULL'
    ids = ids[useful]
    tweets = tweets[useful]
    label_b = label_b[useful]

    nums = len(label_b)
    # Tokenize
    # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    token_ids = [tokenizer.encode(text=tweets[i], add_special_tokens=True, max_length=truncate) for i in range(nums)]
    # Get mask
    mask = np.array(get_mask(token_ids))
    # Get lengths
    lens = get_lens(token_ids)
    # Pad tokens
    token_ids = np.array(pad_sents(token_ids, tokenizer.pad_token_id))

    return ids, token_ids, lens, mask, label_b

def task_c(filepath: str, tokenizer, truncate=512):
    nums, ids, tweets, _, _, label_c = read_file(filepath)
    # Only part of the tweets are useful for task c
    useful = label_c != 'NULL'
    ids = ids[useful]
    tweets = tweets[useful]
    label_c = label_c[useful]
    nums = len(label_c)
    # Tokenize
    # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    token_ids = [tokenizer.encode(text=tweets[i], add_special_tokens=True, max_length=truncate) for i in range(nums)]
    # Get mask
    mask = np.array(get_mask(token_ids))
    # Get lengths
    lens = get_lens(token_ids)
    # Pad tokens
    token_ids = np.array(pad_sents(token_ids, tokenizer.pad_token_id))

    return ids, token_ids, lens, mask, label_c

def all_tasks(filepath: str, tokenizer, truncate=512):
    nums, ids, tweets, label_a, label_b, label_c = read_file(filepath)
    # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    token_ids = [tokenizer.encode(text=tweets[i], add_special_tokens=True, max_length=truncate) for i in range(nums)]
    mask = np.array(get_mask(token_ids))
    lens = get_lens(token_ids)
    token_ids = np.array(pad_sents(token_ids, tokenizer.pad_token_id))

    return ids, token_ids, lens, mask, label_a, label_b, label_c

Data preprocessing


In [ ]:
def emoji2word(sents):
    return [emoji.demojize(sent) for sent in sents]

def remove_useless_punctuation(sents):
    for i, sent in enumerate(sents):
        sent = sent.replace(':', ' ')
        sent = sent.replace('_', ' ')
        sent = sent.replace('...', ' ')
        sents[i] = sent
    return sents

def remove_replicates(sents):
    # if there are multiple `@USER` tokens in a tweet, replace it with `@USERS`
    # because some tweets contain so many `@USER` which may cause redundant
    for i, sent in enumerate(sents):
        if sent.find('@USER') != sent.rfind('@USER'):
            sents[i] = sent.replace('@USER', '')
            sents[i] = '@USERS ' + sents[i]
    return sents

def replace_rare_words(sents):
    rare_words = {
        'URL': 'http'
    }
    for i, sent in enumerate(sents):
        for w in rare_words.keys():
            sents[i] = sent.replace(w, rare_words[w])
    return sents

def segment_hashtag(sents):
    # E.g. '#LunaticLeft' => 'lunatic left'
    for i, sent in enumerate(sents):
        sent_tokens = sent.split(' ')
        for j, t in enumerate(sent_tokens):
            if t.find('#') == 0:
                sent_tokens[j] = ' '.join(segment(t))
        sents[i] = ' '.join(sent_tokens)
    return sents


def process_tweets(tweets):
    # Process tweets
    tweets = emoji2word(tweets)
    tweets = replace_rare_words(tweets)
    tweets = remove_replicates(tweets)
    tweets = segment_hashtag(tweets)
    tweets = remove_useless_punctuation(tweets)
    tweets = np.array(tweets)
    return tweets

Dataset

In [ ]:
class HuggingfaceDataset(Dataset):
    def __init__(self, input_ids, lens, mask, labels, task):
        self.input_ids = torch.tensor(input_ids)
        self.lens = lens
        self.mask = torch.tensor(mask, dtype=torch.float32)
        self.labels = labels
        self.task = task

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        this_LABEL_DICT = LABEL_DICT[self.task]
        input = self.input_ids[idx]
        length = self.lens[idx]
        mask = self.mask[idx]
        label = torch.tensor(this_LABEL_DICT[self.labels[idx]])
        return input, length, mask, label

class HuggingfaceMTDataset(Dataset):
    def __init__(self, input_ids, lens, mask, labels, task):
        self.input_ids = torch.tensor(input_ids)
        self.lens = lens
        self.mask = torch.tensor(mask, dtype=torch.float32)
        self.labels = labels

    def __len__(self):
        return self.labels['a'].shape[0]

    def __getitem__(self, idx):
        input = self.input_ids[idx]
        mask = self.mask[idx]
        length = self.lens[idx]
        label_A = torch.tensor(LABEL_DICT['a'][self.labels['a'][idx]])
        label_B = torch.tensor(LABEL_DICT['b'][self.labels['b'][idx]])
        label_C = torch.tensor(LABEL_DICT['c'][self.labels['c'][idx]])
        return input, length, mask, label_A, label_B, label_C

class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """
    Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
    """

    def __init__(self, dataset, indices=None, num_samples=None):
        # if indices is not provided,
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset.labels))) \
            if indices is None else indices

        # if num_samples is not provided,
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples

        # distribution of classes in the dataset
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1

        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)] for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, id_):
        return dataset.labels[id_]

    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

Model

In [ ]:
#BERT
class BERT(nn.Module):
    def __init__(self, model_size, num_labels=2):
        super(BERT, self).__init__()
        self.model = BertForSequenceClassification.from_pretrained(
            f'bert-{model_size}-uncased',
            num_labels=num_labels,
            hidden_dropout_prob= 0.1,
            attention_probs_dropout_prob= 0.1
        )

        # Freeze embeddings' parameters for saving memory
        # for param in self.model.bert.embeddings.parameters():
        #     param.requires_grad = False

    def forward(self, inputs, lens, mask, labels=None):
        outputs = self.model(inputs, attention_mask=mask)
        logits = outputs[0]
        # return loss, logits
        return logits

class RoBERTa(nn.Module):
    def __init__(self, model_size, num_labels=2):
        super(RoBERTa, self).__init__()
        self.model = RobertaForSequenceClassification.from_pretrained(
            f'roberta-{model_size}',
            num_labels=num_labels,
            hidden_dropout_prob= 0.1,
            attention_probs_dropout_prob= 0.1
        )

        # Freeze embeddings' parameters for saving memory
        # for param in self.model.roberta.embeddings.parameters():
        #     param.requires_grad = False

    def forward(self, inputs, lens, mask, labels):
        outputs = self.model(inputs, attention_mask=mask, labels=labels)
        loss, logits = outputs[:2]
        # return loss, logits
        return logits

class MTModel(nn.Module):
    def __init__(self, model, model_size):
        super(MTModel, self).__init__()
        if model == 'bert':
            pretrained = BertForSequenceClassification.from_pretrained(
                f'bert-{model_size}-uncased',
                hidden_dropout_prob= 0.1,
                attention_probs_dropout_prob= 0.1
            )
            self.main = pretrained.bert
            self.dropout = pretrained.dropout
        elif model == 'roberta':
            pretrained = RobertaForSequenceClassification.from_pretrained(
                f'roberta-{model_size}',
                hidden_dropout_prob= 0.1,
                attention_probs_dropout_prob= 0.1
            )
            self.main = pretrained.roberta
            self.dropout = pretrained.dropout

        # Freeze embeddings' parameters for saving memory
        for param in self.main.embeddings.parameters():
            param.requires_grad = False

        linear_in_features = 768 if model_size == 'base' else 1024

        self.classifier_a = nn.Linear(in_features=linear_in_features, out_features=2, bias=True)
        self.classifier_b = nn.Linear(in_features=linear_in_features, out_features=3, bias=True)
        self.classifier_c = nn.Linear(in_features=linear_in_features, out_features=4, bias=True)

    def forward(self, inputs, lens, mask):
        outputs = self.main(inputs, attention_mask=mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        # logits for 3 sub-tasks
        logits_A = self.classifier_a(pooled_output)
        logits_B = self.classifier_b(pooled_output)
        logits_C = self.classifier_c(pooled_output)
        return logits_A, logits_B, logits_C

class BERT_LSTM(nn.Module):
    def __init__(self, model_size, num_labels):
        super(BERT_LSTM, self).__init__()
        hidden_size = 300
        self.concat = 'concat'
        input_size = 768 if model_size == 'base' else 1024

        self.emb = BertModel.from_pretrained(
            f'bert-{model_size}-uncased',
            hidden_dropout_prob= 0.1,
            attention_probs_dropout_prob= 0.1
        )

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers= 1,
            bidirectional=True,
            batch_first=True,
            dropout= 0.1 if num_layers > 1 else 0
        )
        self.dropout = nn.Dropout(p= dropout)
        self.linear = nn.Linear(in_features=hidden_size * 2 if self.concat else hidden_size, out_features=num_labels)

    def forward(self, inputs, lens, mask, labels):
        embs = self.emb(inputs, attention_mask=mask)[0] # (batch_size, sequence_length, hidden_size)
        _, (h_n, _) = self.lstm(input=embs) # (num_layers * num_directions, batch, hidden_size)
        if self.concat:
            h_n = torch.cat((h_n[0], h_n[1]), dim=1)
        else:
            h_n = h_n[0] + h_n[1]
        h_n = self.dropout(h_n)
        logits = self.linear(h_n)
        return logits

In [ ]:
#MTL
class MTL_Transformer_LSTM(nn.Module):
    def __init__(self, model, model_size):
        super(MTL_Transformer_LSTM, self).__init__()
        hidden_size = 300
        self.concat = 'concat'
        input_size = 768 if model_size == 'base' else 1024

        if model == 'bert':
            MODEL = BertModel
            model_full_name = f'{model}-{model_size}-uncased'
        elif model == 'roberta':
            MODEL = RobertaModel
            model_full_name = f'{model}-{model_size}'

        self.emb = MODEL.from_pretrained(
            model_full_name,
            hidden_dropout_prob= 0.1,
            attention_probs_dropout_prob= 0.1
        )

        self.LSTMs = nn.ModuleDict({
            'a': nn.LSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers= 1,
                bidirectional=True,
                batch_first=True,
                dropout= 0.1 if num_layers > 1 else 0
            ),
            'b': nn.LSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers= 1,
                bidirectional=True,
                batch_first=True,
                dropout= 0.1 if num_layers > 1 else 0
            ),
            'c': nn.LSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers= 1,
                bidirectional=True,
                batch_first=True,
                dropout= 0.1 if num_layers > 1 else 0
            )
        })

        self.attention_layers = nn.ModuleDict({
            'a': Attention(hidden_size * 2),
            'b': Attention(hidden_size * 2),
            'c': Attention(hidden_size * 2)
        })

        self.dropout = nn.Dropout(p= dropout)

        linear_in_features = hidden_size * 2 if self.concat else hidden_size
        self.Linears = nn.ModuleDict({
            'a': nn.Sequential(
                nn.Linear(linear_in_features, hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size, 2)
            ),
            'b': nn.Sequential(
                nn.Linear(linear_in_features, hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size, 3)
            ),
            'c': nn.Sequential(
                nn.Linear(linear_in_features, hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size, 4)
            )
        })

    def forward(self, inputs, lens, mask):
        embs = self.emb(inputs, attention_mask=mask)[0] # (batch_size, sequence_length, hidden_size)

        _, (h_a, _) = self.LSTMs['a'](embs)
        if self.concat:
            h_a = torch.cat((h_a[0], h_a[1]), dim=1)
        else:
            h_a = h_a[0] + h_a[1]
        h_a = self.dropout(h_a)

        _, (h_b, _) = self.LSTMs['b'](embs)
        if self.concat:
            h_b = torch.cat((h_b[0], h_b[1]), dim=1)
        else:
            h_b = h_b[0] + h_b[1]
        h_b = self.dropout(h_b)

        _, (h_c, _) = self.LSTMs['c'](embs)
        if self.concat:
            h_c = torch.cat((h_c[0], h_c[1]), dim=1)
        else:
            h_c = h_c[0] + h_c[1]
        h_c = self.dropout(h_c)

        logits_a = self.Linears['a'](h_a)
        logits_b = self.Linears['b'](h_b)
        logits_c = self.Linears['c'](h_c)

        return logits_a, logits_b, logits_c

In [ ]:
#gated
class GatedModel(nn.Module):
    def __init__(self, model, model_size):
        super(GatedModel, self).__init__()
        # using BERT/RoBERTa pre-trained model
        if model == 'bert':
            pretrainedA = BertForSequenceClassification.from_pretrained(f'bert-{model_size}-uncased')
            self.mainA = pretrainedA.bert
            pretrainedB = BertForSequenceClassification.from_pretrained(f'bert-{model_size}-uncased')
            self.mainB = pretrainedB.bert
            pretrainedC = BertForSequenceClassification.from_pretrained(f'bert-{model_size}-uncased')
            self.mainC = pretrainedC.bert

            # Freeze embeddings' parameters for saving memory
            for p in [
                # *self.model.robe
                *self.mainA.embeddings.word_embeddings.parameters(),
                *self.mainB.embeddings.word_embeddings.parameters(),
                *self.mainC.embeddings.word_embeddings.parameters(),
            ]:
                p.requires_grad = False

            self.dropoutA = pretrainedA.dropout
            self.dropoutB = pretrainedB.dropout
            self.dropoutC = pretrainedC.dropout
            if model_size == 'base':
                self.hidden_size = 768
            if model_size == 'large':
                self.hidden_size = 1024

            self.linearA = nn.Linear(in_features=self.hidden_size*3, out_features=self.hidden_size, bias=True)
            self.linearB = nn.Linear(in_features=self.hidden_size*3, out_features=self.hidden_size, bias=True)
            self.linearC = nn.Linear(in_features=self.hidden_size*3, out_features=self.hidden_size, bias=True)

            self.softmaxA = nn.Softmax(dim=1)
            self.softmaxB = nn.Softmax(dim=1)
            self.softmaxC = nn.Softmax(dim=1)

            self.classifier_a = nn.Linear(in_features=self.hidden_size, out_features=2, bias=True)
            self.classifier_b = nn.Linear(in_features=self.hidden_size, out_features=3, bias=True)
            self.classifier_c = nn.Linear(in_features=self.hidden_size, out_features=4, bias=True)

        elif model == 'roberta':
            pretrainedA = RobertaForSequenceClassification.from_pretrained(f'roberta-{model_size}')
            self.mainA = pretrainedA.roberta
            pretrainedB = RobertaForSequenceClassification.from_pretrained(f'roberta-{model_size}')
            self.mainB = pretrainedB.roberta
            pretrainedC = RobertaForSequenceClassification.from_pretrained(f'roberta-{model_size}')
            self.mainC = pretrainedC.roberta

            # Freeze embeddings' parameters for saving memory
            for p in [
                # *self.model.robe
                *self.mainA.embeddings.word_embeddings.parameters(),
                *self.mainB.embeddings.word_embeddings.parameters(),
                *self.mainC.embeddings.word_embeddings.parameters(),
            ]:
                p.requires_grad = False

            self.dropoutA = pretrainedA.classifier
            self.dropoutB = pretrainedB.classifier
            self.dropoutC = pretrainedC.classifier

            if model_size == 'base':
                self.hidden_size = 768
            if model_size == 'large':
                self.hidden_size = 1024

            self.linearA = nn.Linear(in_features=self.hidden_size*3, out_features=self.hidden_size, bias=True)
            self.linearB = nn.Linear(in_features=self.hidden_size*3, out_features=self.hidden_size, bias=True)
            self.linearC = nn.Linear(in_features=self.hidden_size*3, out_features=self.hidden_size, bias=True)

            self.softmaxA = nn.Softmax(dim=1)
            self.softmaxB = nn.Softmax(dim=1)
            self.softmaxC = nn.Softmax(dim=1)

            self.classifier_a = nn.Linear(in_features=self.hidden_size, out_features=2, bias=True)
            self.classifier_b = nn.Linear(in_features=self.hidden_size, out_features=3, bias=True)
            self.classifier_c = nn.Linear(in_features=self.hidden_size, out_features=4, bias=True)

    def forward(self, inputs, lens, mask):
        outputsA = self.mainA(inputs, attention_mask=mask)
        pooled_outputA = outputsA[1]
        # pooled_outputA = self.dropoutA(pooled_outputA) # batch_size * hidden_size

        outputsB = self.mainB(inputs, attention_mask=mask)
        pooled_outputB = outputsB[1]
        # pooled_outputB = self.dropoutB(pooled_outputB) # batch_size * hidden_size

        outputsC = self.mainC(inputs, attention_mask=mask)
        pooled_outputC = outputsC[1]
        # pooled_outputC = self.dropoutC(pooled_outputC) # batch_size * hidden_size

        gateA = self.softmaxA(self.linearA(torch.cat((pooled_outputA, pooled_outputB, pooled_outputC), 1)))
        gateB = self.softmaxB(self.linearB(torch.cat((pooled_outputA, pooled_outputB, pooled_outputC), 1)))
        gateC = self.softmaxC(self.linearC(torch.cat((pooled_outputA, pooled_outputB, pooled_outputC), 1)))

        gateA_1 = torch.ones(gateA.shape).cuda() - gateA
        gateB_1 = torch.ones(gateB.shape).cuda() - gateB
        gateC_1 = torch.ones(gateC.shape).cuda() - gateC

        hidden_A = torch.mul(gateA_1, pooled_outputA) + torch.mul(torch.mul(gateA, 0.5), pooled_outputB) + torch.mul(torch.mul(gateA, 0.5), pooled_outputC)
        hidden_B = torch.mul(gateB_1, pooled_outputA) + torch.mul(torch.mul(gateB, 0.5), pooled_outputB) + torch.mul(torch.mul(gateB, 0.5), pooled_outputC)
        hidden_C = torch.mul(gateC_1, pooled_outputA) + torch.mul(torch.mul(gateC, 0.5), pooled_outputB) + torch.mul(torch.mul(gateC, 0.5), pooled_outputC)

        # logits for 3 sub-tasks
        logits_A = self.classifier_a(hidden_A)
        logits_B = self.classifier_b(hidden_B)
        logits_C = self.classifier_c(hidden_C)

        return logits_A, logits_B, logits_C

In [ ]:
task = 'a'
model_name = 'bert'
model_size = 'base'
truncate = 512 #100 #512
epochs = 10
lr = 1e-5 # == 0.00001~0.00007
wd = 0.0
bs = 32 #128 #180 #72
patience = 5
num_layers= 1
dropout = 0.1

In [ ]:
# Fix seed for reproducibility
seed = 19951126
torch.manual_seed(seed)
np.random.seed(seed)


In [ ]:
# Set device
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
num_labels = 3 if task == 'c' else 2

cuda


In [ ]:
#run model and choose tokenzier


if model_name == 'bert':
        if task == 'all':
            model = MTL_Transformer_LSTM(model_name, model_size)
        else:
            model = BERT(model_size, num_labels=num_labels)
        tokenizer = BertTokenizer.from_pretrained(f'bert-{model_size}-uncased')
elif model_name == 'roberta':
        if task == 'all':
            model = MTL_Transformer_LSTM(model_name, model_size)
        else:
            model = RoBERTa(model_size, num_labels=num_labels)
        tokenizer = RobertaTokenizer.from_pretrained(f'roberta-{model_size}')
elif model_name == 'bert-gate' and task == 'all':
        model_name = model_name.replace('-gate', '')
        model = GatedModel(model_name, model_size)
        tokenizer = BertTokenizer.from_pretrained(f'bert-{model_size}-uncased')
elif model_name == 'roberta-gate' and task == 'all':
        model_name = model_name.replace('-gate', '')
        model = GatedModel(model_name, model_size)
        tokenizer = RobertaTokenizer.from_pretrained(f'roberta-{model_size}')

# Move model to correct device
model = model.to(device)

In [ ]:
if task in ['a', 'b', 'c']:
        data_methods = {'a': task_a, 'b': task_b, 'c': task_c}
        ids, token_ids, lens, mask, labels = data_methods[task](TRAIN_PATH, tokenizer=tokenizer, truncate=truncate)
        test_ids, test_token_ids, test_lens, test_mask, test_labels = read_test_file(task, tokenizer=tokenizer, truncate=truncate)
        _Dataset = HuggingfaceDataset
elif task in ['all']:
        ids, token_ids, lens, mask, label_a, label_b, label_c = all_tasks(TRAIN_PATH, tokenizer=tokenizer, truncate=truncate)
        test_ids, test_token_ids, test_lens, test_mask, test_label_a, test_label_b, test_label_c = read_test_file_all(tokenizer)
        labels = {'a': label_a, 'b': label_b, 'c': label_c}
        test_labels = {'a': test_label_a, 'b': test_label_b, 'c': test_label_c}
        _Dataset = HuggingfaceMTDataset


Trainning and Predicting

In [ ]:
# Built-in libraries
import copy
import datetime
from typing import Dict, List
# Third-party libraries


from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from tqdm import tqdm

from torch.utils.data import Dataset
from tqdm import tqdm
import csv

In [ ]:
class Trainer():
    '''
    The trainer for training models.
    It can be used for both single and multi task training.
    Every class function ends with _m is for multi-task training.
    '''
    def __init__(
        self,
        model: nn.Module,
        epochs: int,
        dataloaders: Dict[str, DataLoader],
        criterion: nn.Module,
        loss_weights: List[float],
        clip: bool,
        optimizer: torch.optim.Optimizer,
        scheduler: torch.optim.lr_scheduler,
        device: str,
        patience: int,
        task_name: str,
        model_name: str,
        seed: int
    ):
        self.model = model
        self.epochs = epochs
        self.dataloaders = dataloaders
        self.criterion = criterion
        self.loss_weights = loss_weights
        self.clip = clip
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.patience = patience
        self.task_name = task_name
        self.model_name = model_name
        self.seed = seed
        self.datetimestr = datetime.datetime.now().strftime('%Y-%b-%d_%H:%M:%S')

        # Evaluation results
        self.train_losses = []
        self.test_losses = []
        self.train_f1 = []
        self.test_f1 = []
        self.best_train_f1 = 0.0
        self.best_test_f1 = 0.0

        # Evaluation results for multi-task
        self.best_train_f1_m = np.array([0, 0, 0], dtype=np.float64)
        self.best_test_f1_m = np.array([0, 0, 0], dtype=np.float64)

    def train(self):
        for epoch in range(self.epochs):
            print(f'Epoch {epoch}')
            print('=' * 20)
            self.train_one_epoch()
            self.test()
            print(f'Best test f1: {self.best_test_f1:.4f}')
            print('=' * 20)
            
            if epoch%3 == 0:
              print('Saving results ...')
              save(
                  (self.train_losses, self.test_losses, self.train_f1, self.test_f1, self.best_train_f1, self.best_test_f1),
                  f'./drive/MyDrive/OLID/save/results/single_{self.task_name}_{self.datetimestr}_{self.best_test_f1:.4f}.pt'
              )

    def train_one_epoch(self):
        self.model.train()
        dataloader = self.dataloaders['train']
        y_pred_all = None
        labels_all = None
        loss = 0
        iters_per_epoch = 0
        for inputs, lens, mask, labels in tqdm(dataloader, desc='Training'):
            iters_per_epoch += 1

            if labels_all is None:
                labels_all = labels.numpy()
            else:
                labels_all = np.concatenate((labels_all, labels.numpy()))

            inputs = inputs.to(device=self.device)
            lens = lens.to(device=self.device)
            mask = mask.to(device=self.device)
            labels = labels.to(device=self.device)

            self.optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                # Forward
                logits = self.model(inputs, lens, mask, labels)
                _loss = self.criterion(logits, labels)
                loss += _loss.item()
                y_pred = logits.argmax(dim=1).cpu().numpy()

                if y_pred_all is None:
                    y_pred_all = y_pred
                else:
                    y_pred_all = np.concatenate((y_pred_all, y_pred))

                # Backward
                _loss.backward()
                if self.clip:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=10)
                self.optimizer.step()
                if self.scheduler is not None:
                    self.scheduler.step()

            loss /= iters_per_epoch
            f1 = f1_score(labels_all, y_pred_all, average='macro')

            print(f'loss = {loss:.4f}')
            print(f'Macro-F1 = {f1:.4f}')

            self.train_losses.append(loss)
            self.train_f1.append(f1)
            if f1 > self.best_train_f1:
                self.best_train_f1 = f1

    def test(self):
        self.model.eval()
        dataloader = self.dataloaders['test']
        y_pred_all = None
        labels_all = None
        loss = 0
        iters_per_epoch = 0
        for inputs, lens, mask, labels in tqdm(dataloader, desc='Testing'):
            iters_per_epoch += 1

            if labels_all is None:
                labels_all = labels.numpy()
            else:
                labels_all = np.concatenate((labels_all, labels.numpy()))

            inputs = inputs.to(device=self.device)
            lens = lens.to(device=self.device)
            mask = mask.to(device=self.device)
            labels = labels.to(device=self.device)

            with torch.set_grad_enabled(False):
                logits = self.model(inputs, lens, mask, labels)
                _loss = self.criterion(logits, labels)
                y_pred = logits.argmax(dim=1).cpu().numpy()
                loss += _loss.item()

                if y_pred_all is None:
                    y_pred_all = y_pred
                else:
                    y_pred_all = np.concatenate((y_pred_all, y_pred))

            loss /= iters_per_epoch
            f1 = f1_score(labels_all, y_pred_all, average='macro')

            print(f'loss = {loss:.4f}')
            print(f'Macro-F1 = {f1:.4f}')

            self.test_losses.append(loss)
            self.test_f1.append(f1)
            if f1 > self.best_test_f1:
                self.best_test_f1 = f1
                self.save_model()

    def train_m(self):
        for epoch in range(self.epochs):
            print(f'Epoch {epoch}')
            print('=' * 20)
            self.train_one_epoch_m()
            self.test_m()
            print(f'Best test results A: {self.best_test_f1_m[0]:.4f}')
            print(f'Best test results B: {self.best_test_f1_m[1]:.4f}')
            print(f'Best test results C: {self.best_test_f1_m[2]:.4f}')
            print('=' * 20)

        print('Saving results ...')
        save(
            (self.train_losses, self.test_losses, self.train_f1, self.test_f1, self.best_train_f1_m, self.best_test_f1_m),
            f'./drive/MyDrive/OLID/save/results/mtl_{self.datetimestr}_{self.best_test_f1_m[0]:.4f}.pt'
        )

    def train_one_epoch_m(self):
        self.model.train()
        dataloader = self.dataloaders['train']

        y_pred_all_A = None
        y_pred_all_B = None
        y_pred_all_C = None
        labels_all_A = None
        labels_all_B = None
        labels_all_C = None

        loss = 0
        iters_per_epoch = 0
        for inputs, lens, mask, label_A, label_B, label_C in tqdm(dataloader, desc='Training M'):
            iters_per_epoch += 1

            inputs = inputs.to(device=self.device)
            lens = lens.to(device=self.device)
            mask = mask.to(device=self.device)
            label_A = label_A.to(device=self.device)
            label_B = label_B.to(device=self.device)
            label_C = label_C.to(device=self.device)

            self.optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                # Forward
                # logits_A, logits_B, logits_C = self.model(inputs, mask)
                all_logits = self.model(inputs, lens, mask)
                y_pred_A = all_logits[0].argmax(dim=1).cpu().numpy()
                y_pred_B = all_logits[1][:, 0:2].argmax(dim=1)
                y_pred_C = all_logits[2][:, 0:3].argmax(dim=1)

                Non_null_index_B = label_B != LABEL_DICT['b']['NULL']
                Non_null_label_B = label_B[Non_null_index_B]
                Non_null_pred_B = y_pred_B[Non_null_index_B]

                Non_null_index_C = label_C != LABEL_DICT['c']['NULL']
                Non_null_label_C = label_C[Non_null_index_C]
                Non_null_pred_C = y_pred_C[Non_null_index_C]

                labels_all_A = label_A.cpu().numpy() if labels_all_A is None else np.concatenate((labels_all_A, label_A.cpu().numpy()))
                labels_all_B = Non_null_label_B.cpu().numpy() if labels_all_B is None else np.concatenate((labels_all_B, Non_null_label_B.cpu().numpy()))
                labels_all_C = Non_null_label_C.cpu().numpy() if labels_all_C is None else np.concatenate((labels_all_C, Non_null_label_C.cpu().numpy()))

                y_pred_all_A = y_pred_A if y_pred_all_A is None else np.concatenate((y_pred_all_A, y_pred_A))
                y_pred_all_B = Non_null_pred_B.cpu().numpy() if y_pred_all_B is None else np.concatenate((y_pred_all_B, Non_null_pred_B.cpu().numpy()))
                y_pred_all_C = Non_null_pred_C.cpu().numpy() if y_pred_all_C is None else np.concatenate((y_pred_all_C, Non_null_pred_C.cpu().numpy()))

                # f1[0] += self.calc_f1(label_A, y_pred_A)
                # f1[1] += self.calc_f1(Non_null_label_B, Non_null_pred_B)
                # f1[2] += self.calc_f1(Non_null_label_C, Non_null_pred_C)

                _loss = self.loss_weights[0] * self.criterion(all_logits[0], label_A)
                _loss += self.loss_weights[1] * self.criterion(all_logits[1], label_B)
                _loss += self.loss_weights[2] * self.criterion(all_logits[2], label_C)
                loss += _loss.item()

                # Backward
                _loss.backward()
                if self.clip:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=10)
                self.optimizer.step()
                if self.scheduler is not None:
                    self.scheduler.step()

        loss /= iters_per_epoch
        f1_A = f1_score(labels_all_A, y_pred_all_A, average='macro')
        f1_B = f1_score(labels_all_B, y_pred_all_B, average='macro')
        f1_C = f1_score(labels_all_C, y_pred_all_C, average='macro')

        print(f'loss = {loss:.4f}')
        print(f'A: {f1_A:.4f}')
        print(f'B: {f1_B:.4f}')
        print(f'C: {f1_C:.4f}')

        self.train_losses.append(loss)
        self.train_f1.append([f1_A, f1_B, f1_C])

        if f1_A > self.best_train_f1_m[0]:
            self.best_train_f1_m[0] = f1_A
        if f1_B > self.best_train_f1_m[1]:
            self.best_train_f1_m[1] = f1_B
        if f1_C > self.best_train_f1_m[2]:
            self.best_train_f1_m[2] = f1_C

    def test_m(self):
        self.model.eval()
        dataloader = self.dataloaders['test']
        loss = 0
        iters_per_epoch = 0

        y_pred_all_A = None
        y_pred_all_B = None
        y_pred_all_C = None
        labels_all_A = None
        labels_all_B = None
        labels_all_C = None

        for inputs, lens, mask, label_A, label_B, label_C in tqdm(dataloader, desc='Test M'):
            iters_per_epoch += 1

            labels_all_A = label_A.numpy() if labels_all_A is None else np.concatenate((labels_all_A, label_A.numpy()))
            labels_all_B = label_B.numpy() if labels_all_B is None else np.concatenate((labels_all_B, label_B.numpy()))
            labels_all_C = label_C.numpy() if labels_all_C is None else np.concatenate((labels_all_C, label_C.numpy()))

            inputs = inputs.to(device=self.device)
            lens = lens.to(device=self.device)
            mask = mask.to(device=self.device)
            label_A = label_A.to(device=self.device)
            label_B = label_B.to(device=self.device)
            label_C = label_C.to(device=self.device)

            with torch.set_grad_enabled(False):
                all_logits = self.model(inputs, lens, mask)
                y_pred_A = all_logits[0].argmax(dim=1).cpu().numpy()
                y_pred_B = all_logits[1].argmax(dim=1).cpu().numpy()
                y_pred_C = all_logits[2].argmax(dim=1).cpu().numpy()

                # f1[0] += self.calc_f1(label_A, y_pred_A)
                # f1[1] += self.calc_f1(label_B, y_pred_B)
                # f1[2] += self.calc_f1(label_C, y_pred_C)

                y_pred_all_A = y_pred_A if y_pred_all_A is None else np.concatenate((y_pred_all_A, y_pred_A))
                y_pred_all_B = y_pred_B if y_pred_all_B is None else np.concatenate((y_pred_all_B, y_pred_B))
                y_pred_all_C = y_pred_C if y_pred_all_C is None else np.concatenate((y_pred_all_C, y_pred_C))

                _loss = self.loss_weights[0] * self.criterion(all_logits[0], label_A)
                _loss += self.loss_weights[1] * self.criterion(all_logits[1], label_B)
                _loss += self.loss_weights[2] * self.criterion(all_logits[2], label_C)
                loss += _loss.item()

        loss /= iters_per_epoch
        f1_A = f1_score(labels_all_A, y_pred_all_A, average='macro')
        f1_B = f1_score(labels_all_B, y_pred_all_B, average='macro')
        f1_C = f1_score(labels_all_C, y_pred_all_C, average='macro')

        print(f'loss = {loss:.4f}')
        print(f'A: {f1_A:.4f}')
        print(f'B: {f1_B:.4f}')
        print(f'C: {f1_C:.4f}')

        self.test_losses.append(loss)
        self.test_f1.append([f1_A, f1_B, f1_C])

        if f1_A > self.best_test_f1_m[0]:
            self.best_test_f1_m[0] = f1_A
            self.save_model()
        if f1_B > self.best_test_f1_m[1]:
            self.best_test_f1_m[1] = f1_B
        if f1_C > self.best_test_f1_m[2]:
            self.best_test_f1_m[2] = f1_C

        # for i in range(len(f1)):
        #     for j in range(len(f1[0])):
        #         if f1[i][j] > self.best_test_f1_m[i][j]:
        #             self.best_test_f1_m[i][j] = f1[i][j]
        #             if i == 0 and j == 0:
        #                 self.save_model()

    def calc_f1(self, labels, y_pred):
        return np.array([
            f1_score(labels.cpu(), y_pred.cpu(), average='macro'),
            f1_score(labels.cpu(), y_pred.cpu(), average='micro'),
            f1_score(labels.cpu(), y_pred.cpu(), average='weighted')
        ], np.float64)

    def printing(self, loss, f1):
        print(f'loss = {loss:.4f}')
        print(f'Macro-F1 = {f1[0]:.4f}')
        # print(f'Micro-F1 = {f1[1]:.4f}')
        # print(f'Weighted-F1 = {f1[2]:.4f}')

    def save_model(self):
        print('Saving model...')
        if self.task_name == 'all':
            filename = f'./drive/MyDrive/OLID/save/models/{self.task_name}_{self.model_name}_{self.best_test_f1_m[0]}_seed{self.seed}.pt'
        else:
            filename = f'./drive/MyDrive/OLID/save/models/{self.task_name}_{self.model_name}_{self.best_test_f1}_seed{self.seed}.pt'
        save(copy.deepcopy(self.model.state_dict()), filename)

In [ ]:
datasets = {
        'train': _Dataset(
            input_ids=token_ids,
            lens=lens,
            mask=mask,
            labels=labels,
            task=task
        ),
        'test': _Dataset(
            input_ids=test_token_ids,
            lens=test_lens,
            mask=test_mask,
            labels=test_labels,
            task=task
        )
    }

sampler = ImbalancedDatasetSampler(datasets['train']) if task in ['a', 'b', 'c'] else None
dataloaders = {
    'train': DataLoader(
        dataset=datasets['train'],
        batch_size=bs,
        sampler=sampler
    ),
    'test': DataLoader(dataset=datasets['test'], batch_size=bs)
    }

criterion = torch.nn.CrossEntropyLoss()

if True:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        # A warmup scheduler
        t_total = epochs * len(dataloaders['train'])
        warmup_steps = np.ceil(t_total / 10.0) * 2
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=t_total
        )
else:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        scheduler = None




In [ ]:
trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,        criterion=criterion,
        loss_weights=[0.4, 0.3, 0.3],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )

if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
task = 'a'
model_name = 'bert'
model_size = 'base'
truncate = 512 #100 #512
epochs = 10
lr = 1e-4 # == 0.00001~0.00007
wd = 0.0
bs = 32 #128 #180 #72
patience = 5
num_layers= 1
dropout = 0.1

datasets = {
        'train': _Dataset(
            input_ids=token_ids,
            lens=lens,
            mask=mask,
            labels=labels,
            task=task
        ),
        'test': _Dataset(
            input_ids=test_token_ids,
            lens=test_lens,
            mask=test_mask,
            labels=test_labels,
            task=task
        )
    }

sampler = ImbalancedDatasetSampler(datasets['train']) if task in ['a', 'b', 'c'] else None
dataloaders = {
    'train': DataLoader(
        dataset=datasets['train'],
        batch_size=bs,
        sampler=sampler
    ),
    'test': DataLoader(dataset=datasets['test'], batch_size=bs)
    }

criterion = torch.nn.CrossEntropyLoss()

if True:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        # A warmup scheduler
        t_total = epochs * len(dataloaders['train'])
        warmup_steps = np.ceil(t_total / 10.0) * 2
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=t_total
        )
else:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        scheduler = None

trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.4, 0.3, 0.3],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )

if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
task = 'a'
model_name = 'bert'
model_size = 'base'
truncate = 512 #100 #512
epochs = 10
lr = 5e-5 # == 0.00001~0.00007
wd = 0.0
bs = 32 #128 #180 #72
patience = 5
num_layers= 1
dropout = 0.1

datasets = {
        'train': _Dataset(
            input_ids=token_ids,
            lens=lens,
            mask=mask,
            labels=labels,
            task=task
        ),
        'test': _Dataset(
            input_ids=test_token_ids,
            lens=test_lens,
            mask=test_mask,
            labels=test_labels,
            task=task
        )
    }

sampler = ImbalancedDatasetSampler(datasets['train']) if task in ['a', 'b', 'c'] else None
dataloaders = {
    'train': DataLoader(
        dataset=datasets['train'],
        batch_size=bs,
        sampler=sampler
    ),
    'test': DataLoader(dataset=datasets['test'], batch_size=bs)
    }

criterion = torch.nn.CrossEntropyLoss()

if True:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        # A warmup scheduler
        t_total = epochs * len(dataloaders['train'])
        warmup_steps = np.ceil(t_total / 10.0) * 2
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=t_total
        )
else:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        scheduler = None

trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.4, 0.3, 0.3],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )

if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
task = 'a'
model_name = 'bert'
model_size = 'base'
truncate = 512 #100 #512
epochs = 10
lr = 2e-5 # == 0.00001~0.00007
wd = 0.0
bs = 32 #128 #180 #72
patience = 5
num_layers= 1
dropout = 0.1

datasets = {
        'train': _Dataset(
            input_ids=token_ids,
            lens=lens,
            mask=mask,
            labels=labels,
            task=task
        ),
        'test': _Dataset(
            input_ids=test_token_ids,
            lens=test_lens,
            mask=test_mask,
            labels=test_labels,
            task=task
        )
    }

sampler = ImbalancedDatasetSampler(datasets['train']) if task in ['a', 'b', 'c'] else None
dataloaders = {
    'train': DataLoader(
        dataset=datasets['train'],
        batch_size=bs,
        sampler=sampler
    ),
    'test': DataLoader(dataset=datasets['test'], batch_size=bs)
    }

criterion = torch.nn.CrossEntropyLoss()

if True:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        # A warmup scheduler
        t_total = epochs * len(dataloaders['train'])
        warmup_steps = np.ceil(t_total / 10.0) * 2
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=t_total
        )
else:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        scheduler = None

trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.4, 0.3, 0.3],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )

if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
task = 'a'
model_name = 'bert'
model_size = 'base'
truncate = 512 #100 #512
epochs = 10
lr = 7e-5 # == 0.00001~0.00007
wd = 0.0
bs = 32 #128 #180 #72
patience = 5
num_layers= 1
dropout = 0.1

datasets = {
        'train': _Dataset(
            input_ids=token_ids,
            lens=lens,
            mask=mask,
            labels=labels,
            task=task
        ),
        'test': _Dataset(
            input_ids=test_token_ids,
            lens=test_lens,
            mask=test_mask,
            labels=test_labels,
            task=task
        )
    }

sampler = ImbalancedDatasetSampler(datasets['train']) if task in ['a', 'b', 'c'] else None
dataloaders = {
    'train': DataLoader(
        dataset=datasets['train'],
        batch_size=bs,
        sampler=sampler
    ),
    'test': DataLoader(dataset=datasets['test'], batch_size=bs)
    }

criterion = torch.nn.CrossEntropyLoss()

if True:
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        # A warmup scheduler
        t_total = epochs * len(dataloaders['train'])
        warmup_steps = np.ceil(t_total / 10.0) * 2
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=t_total
        )
else:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        scheduler = None

trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.4, 0.3, 0.3],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )

if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
task = 'b'
model_name = 'bert'
model_size = 'base'
truncate = 512 #100 #512
epochs = 10
lr = 1e-4 # == 0.00001~0.00007
wd = 0.0
bs = 32 #128 #180 #72
patience = 5
num_layers= 1
dropout = 0.1


In [ ]:
trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.4, 0.3, 0.3],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )
if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
task = 'c'
model_name = 'bert'
model_size = 'base'
truncate = 512 #100 #512
epochs = 10
lr = 1e-4 # == 0.00001~0.00007
wd = 0.0
bs = 32 #128 #180 #72
patience = 5
num_layers= 1
dropout = 0.1


In [ ]:
trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.4, 0.3, 0.3],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )
if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
task = 'all'
model_name = 'bert'
model_size = 'base'
truncate = 512 #100 #512
epochs = 30
lr = 1e-4 # == 0.00001~0.00007
wd = 0.0
bs = 32 #128 #180 #72
patience = 5
num_layers= 1
dropout = 0.1


In [ ]:
trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.4, 0.3, 0.3],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )
if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
epochs = 30

trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.5, 0.42, 0.08],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )
if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.6, 0.32, 0.08],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )
if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()

In [ ]:
trainer = Trainer(
        model=model,
        epochs=epochs,
        dataloaders=dataloaders,
        criterion=criterion,
        loss_weights=[0.6, 0.32, 0.08],
        clip= True,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        patience=patience,
        task_name=task,
        model_name=model_name, 
        seed=19951126
    )
if task in ['a', 'b', 'c']:
    trainer.train()
else:
    trainer.train_m()